In [75]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [76]:
import pandas as pd
import numpy as np
from datetime import datetime, date

import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [77]:
spark = SparkSession.builder.getOrCreate()

**1.导入巴黎参数**

In [78]:
psdf_person_paris=ps.read_csv('/Users/ordi_de_lvga/Documents/nf26_td/projet/BDD_BGES/BDD_BGES_PARIS/PERSONNEL_PARIS.txt', sep=';',index_col='ID_PERSONNEL')
psdf_person_paris.head(5)

,NOM_PERSONNEL,PRENOM_PERSONNEL,DT_NAISS,VILLE_NAISS,PAYS_NAISS,NUM_SECU,IND_PAYS_NUM_TELP,NUM_TELEPHONE,NUM_VOIE,DSC_VOIE,CMPL_VOIE,CD_POSTAL,VILLE,PAYS,FONCTION_PERSONNEL,TS_CREATION_PERSONNEL,TS_MAJ_PPERSONNEL
ID_PERSONNEL,,,,,,,,,,,,,,,,,
KeyPers_Paris_1230000,Name0,FistName0,1992-03-13,Lille,France,NS000000000,None,+336##0151713,22,NomVoie100,None,#9423,Paris,France,Ingénieur Informaticien,2005-06-24 06:48:21,2005-06-24 06:48:21
KeyPers_Paris_1230001,Name1,FistName1,1955-02-17,Pekin,China,NS000000001,None,+336##0797149,33,NomVoie731,None,#2429,Paris,France,Ingénieur Data,2007-11-16 22:38:43,2007-11-16 22:38:43
KeyPers_Paris_1230002,Name2,FistName2,1982-10-06,Alger,Algeria,NS000000002,None,+336##0319378,55,NomVoie622,None,#7861,Paris,France,Ingénieur Informaticien,2004-07-09 14:25:16,2004-07-09 14:25:16
KeyPers_Paris_1230003,Name3,FistName3,1955-08-22,Lima,Peru,NS000000003,None,+336##0205027,61,NomVoie363,None,#4028,Paris,France,Ingénieur Data,2013-02-16 23:23:05,2013-02-16 23:23:05
KeyPers_Paris_1230004,Name4,FistName4,2017-04-07,Pekin,China,NS000000004,None,+336##0456031,68,NomVoie914,None,#0111,Paris,France,Cadre,2001-10-13 22:42:51,2001-10-13 22:42:51


In [79]:
psdf_informatique_paris=ps.read_csv('/Users/ordi_de_lvga/Documents/nf26_td/projet/BDD_BGES/BDD_BGES_PARIS/BDD_BGES_PARIS_INFORMATIQUE', sep=';',index_col='ID_MATERIELINFO')
psdf_informatique_paris.head(5)

,ID_PERSONNEL,NOM_PERSONNEL,PRENOM_PERSONNEL,DATE_ACHAT,TYPE,MODELE
ID_MATERIELINFO,,,,,,
Paris_MATERIEL_INFO_202406250,KeyPers_Paris_1233515,Name3515,FistName3515,2024-06-25 11:37:24,Ecran,modèle par défaut
Paris_MATERIEL_INFO_202406251,KeyPers_Paris_1232092,Name2092,FistName2092,2024-06-25 10:48:59,Vidéo projecteur,modèle par défaut
Paris_MATERIEL_INFO_202406252,KeyPers_Paris_1231450,Name1450,FistName1450,2024-06-25 10:52:46,PC fixe sans ecran,"Elite (Tower, SFF, One)"
Paris_MATERIEL_INFO_202406253,KeyPers_Paris_1234268,Name4268,FistName4268,2024-06-25 11:55:34,imprimante,Laser A3 (40-99kg)
Paris_MATERIEL_INFO_202406254,KeyPers_Paris_1230872,Name872,FistName872,2024-06-25 16:44:37,PC portable,Moyenne 14/15pouces


In [80]:
psdf_mission_paris=ps.read_csv('/Users/ordi_de_lvga/Documents/nf26_td/projet/BDD_BGES/BDD_BGES_PARIS/BDD_BGES_PARIS_MISSION', sep=';',index_col='ID_MISSION')
psdf_mission_paris.head(5)

,ID_PERSONNEL,NOM_PERSONNEL,PRENOM_PERSONNEL,DATE_MISSION,TYPE_MISSION,VILLE_DEPART,PAYS_DEPART,VILLE_DESTINATION,PAYS_DESTINATION,TRANSPORT,ALLER_RETOUR
ID_MISSION,,,,,,,,,,,
Paris_202405180,KeyPers_Paris_1233185,Name3185,FistName3185,2024-05-18 23:19:11,Réunion,Paris,France,Wellington,New Zealand,Avion,oui
Paris_202405181,KeyPers_Paris_1230183,Name183,FistName183,2024-05-18 10:54:36,Développement,Paris,France,Sidney,Australia,Avion,oui
Paris_202405182,KeyPers_Paris_1232953,Name2953,FistName2953,2024-05-18 06:43:42,Développement,Paris,France,Auckland,New Zealand,Avion,oui
Paris_202405183,KeyPers_Paris_1234798,Name4798,FistName4798,2024-05-18 22:28:46,Conférence,Paris,France,Oslo,Norvège,Avion,non
Paris_202405184,KeyPers_Paris_1231863,Name1863,FistName1863,2024-05-18 04:16:39,Rencontre entreprises,Paris,France,Lima,Peru,Avion,oui


**2.导入上海数据**

In [81]:
psdf_person_shanghai=ps.read_csv('/Users/ordi_de_lvga/Documents/nf26_td/projet/BDD_BGES/BDD_BGES_SHANGHAI/PERSONNEL_SHANGHAI.txt', sep=';',index_col='ID_PERSONNEL')
psdf_person_shanghai.head(5)

,NOM_PERSONNEL,PRENOM_PERSONNEL,DT_NAISS,VILLE_NAISS,PAYS_NAISS,NUM_SECU,IND_PAYS_NUM_TELP,NUM_TELEPHONE,NUM_VOIE,DSC_VOIE,CMPL_VOIE,CD_POSTAL,VILLE,PAYS,FONCTION_PERSONNEL,TS_CREATION_PERSONNEL,TS_MAJ_PPERSONNEL
ID_PERSONNEL,,,,,,,,,,,,,,,,,
KeyPers_Shanghai_1230000,Name0,FistName0,2001-05-14,Rio de Janeiro,Brazil,NS000000000,None,+336##0481414,86,NomVoie440,None,#4474,Shanghai,China,Business Executive,2015-07-07 10:18:57,2015-07-07 10:18:57
KeyPers_Shanghai_1230001,Name1,FistName1,1995-06-01,Compiègne,France,NS000000001,None,+336##0648537,86,NomVoie661,None,#3428,Shanghai,China,Data Engineer,2015-02-14 08:32:07,2015-02-14 08:32:07
KeyPers_Shanghai_1230002,Name2,FistName2,1983-06-24,Compiègne,France,NS000000002,None,+336##0641720,60,NomVoie722,None,#2672,Shanghai,China,Computer Engineer,2011-03-24 21:17:01,2011-03-24 21:17:01
KeyPers_Shanghai_1230003,Name3,FistName3,1963-05-11,Shanghai,China,NS000000003,None,+336##0405078,43,NomVoie773,None,#5575,Shanghai,China,Data Engineer,2013-02-26 20:56:07,2013-02-26 20:56:07
KeyPers_Shanghai_1230004,Name4,FistName4,1988-10-08,Vancouver,Canada,NS000000004,None,+336##0899659,94,NomVoie594,None,#7804,Shanghai,China,Data Engineer,2004-08-18 06:54:54,2004-08-18 06:54:54


In [82]:
psdf_informatique_shanghai=ps.read_csv('/Users/ordi_de_lvga/Documents/nf26_td/projet/BDD_BGES/BDD_BGES_SHANGHAI/BDD_BGES_SHANGHAI_INFORMATIQUE', sep=';',index_col='ID_MATERIELINFO')
psdf_informatique_shanghai.head(5)

,ID_PERSONNEL,NOM_PERSONNEL,PRENOM_PERSONNEL,DATE_ACHAT,TYPE,MODELE
ID_MATERIELINFO,,,,,,
Shanghai_MATERIEL_INFO_202409200,KeyPers_Shanghai_1230677,Name677,FistName677,2024-09-20 13:48:41,PC portable,modèle par défaut
Shanghai_MATERIEL_INFO_202409201,KeyPers_Shanghai_1230621,Name621,FistName621,2024-09-20 08:39:31,PC fixe sans ecran,modèle par défaut
Shanghai_MATERIEL_INFO_202409202,KeyPers_Shanghai_1230799,Name799,FistName799,2024-09-20 15:09:36,Vidéo projecteur,Pour salle
Shanghai_MATERIEL_INFO_202409203,KeyPers_Shanghai_1230479,Name479,FistName479,2024-09-20 15:02:34,PC portable,EliteBook 6xx
Shanghai_MATERIEL_INFO_202409204,KeyPers_Shanghai_1230452,Name452,FistName452,2024-09-20 13:19:18,,modèle par défaut


In [83]:
psdf_mission_shanghai=ps.read_csv('/Users/ordi_de_lvga/Documents/nf26_td/projet/BDD_BGES/BDD_BGES_SHANGHAI/BDD_BGES_SHANGHAI_MISSION', sep=';',index_col='ID_MISSION')
psdf_mission_shanghai.head(5)

,ID_PERSONNEL,NOM_PERSONNEL,PRENOM_PERSONNEL,DATE_MISSION,TYPE_MISSION,VILLE_DEPART,PAYS_DEPART,VILLE_DESTINATION,PAYS_DESTINATION,TRANSPORT,ALLER_RETOUR
ID_MISSION,,,,,,,,,,,
Shanghai_202406050,KeyPers_Shanghai_1230528,Name528,FistName528,2024-06-05 15:10:01,Development,Shanghai,China,Los Angeles,USA,Avion,oui
Shanghai_202406051,KeyPers_Shanghai_1230794,Name794,FistName794,2024-06-05 08:12:49,Vocational Training,Shanghai,China,Dubaï,Emirats,Avion,oui
Shanghai_202406052,KeyPers_Shanghai_1230681,Name681,FistName681,2024-06-05 09:43:46,Development,Shanghai,China,Pekin,China,Train,oui
Shanghai_202406053,KeyPers_Shanghai_1230807,Name807,FistName807,2024-06-05 15:55:40,Vocational Training,Shanghai,China,Marseille,France,Avion,oui
Shanghai_202406054,KeyPers_Shanghai_1230435,Name435,FistName435,2024-06-05 11:04:37,Conference,Shanghai,China,Alger,Algeria,Avion,non


**3.导入柏林数据**

In [84]:
psdf_person_berlin=ps.read_csv('/Users/ordi_de_lvga/Documents/nf26_td/projet/BDD_BGES/BDD_BGES_BERLIN/PERSONNEL_BERLIN.txt', sep=';',index_col='ID_PERSONNEL')
psdf_person_berlin.head(5)

,NOM_PERSONNEL,PRENOM_PERSONNEL,DT_NAISS,VILLE_NAISS,PAYS_NAISS,NUM_SECU,IND_PAYS_NUM_TELP,NUM_TELEPHONE,NUM_VOIE,DSC_VOIE,CMPL_VOIE,CD_POSTAL,VILLE,PAYS,FONCTION_PERSONNEL,TS_CREATION_PERSONNEL,TS_MAJ_PPERSONNEL
ID_PERSONNEL,,,,,,,,,,,,,,,,,
KeyPers_Berlin_1230000,Name0,FistName0,1993-11-04,Osaka,Japan,NS000000000,None,+336##0263188,57,NomVoie940,None,#8830,Berlin,Germany,Dateningenieur,2010-09-09 11:22:17,2010-09-09 11:22:17
KeyPers_Berlin_1230001,Name1,FistName1,1932-11-22,Wellington,New Zealand,NS000000001,None,+336##0401873,64,NomVoie711,None,#9785,Berlin,Germany,Führungskraft,2017-07-24 16:51:18,2017-07-24 16:51:18
KeyPers_Berlin_1230002,Name2,FistName2,1990-08-12,Sidney,Australia,NS000000002,None,+336##0524126,94,NomVoie322,None,#4816,Berlin,Germany,Dateningenieur,1997-08-20 04:01:46,1997-08-20 04:01:46
KeyPers_Berlin_1230003,Name3,FistName3,1965-05-26,Rabat,Maroc,NS000000003,None,+336##0418484,78,NomVoie593,None,#3546,Berlin,Germany,Ökonom,1998-11-17 19:25:01,1998-11-17 19:25:01
KeyPers_Berlin_1230004,Name4,FistName4,1959-01-18,Shanghai,China,NS000000004,None,+336##0986317,65,NomVoie404,None,#6788,Berlin,Germany,Dateningenieur,2010-11-10 04:24:37,2010-11-10 04:24:37


In [85]:
psdf_informatique_berlin=ps.read_csv('/Users/ordi_de_lvga/Documents/nf26_td/projet/BDD_BGES/BDD_BGES_BERLIN/BDD_BGES_BERLIN_INFORMATIQUE', sep=';',index_col='ID_MATERIELINFO')
psdf_informatique_berlin.head(5)

,ID_PERSONNEL,NOM_PERSONNEL,PRENOM_PERSONNEL,DATE_ACHAT,TYPE,MODELE
ID_MATERIELINFO,,,,,,
BERLIN_MATERIEL_INFO_202411050,KeyPers_Berlin_1233545,Name3545,FistName3545,2024-11-05 12:14:51,PC fixe sans ecran,"Elite (Tower, SFF, One)"
BERLIN_MATERIEL_INFO_202411051,KeyPers_Berlin_1233662,Name3662,FistName3662,2024-11-05 16:09:40,Serveur,Modèle par défaut
BERLIN_MATERIEL_INFO_202411052,KeyPers_Berlin_1231784,Name1784,FistName1784,2024-11-05 15:46:01,souris,modèle par défaut
BERLIN_MATERIEL_INFO_202411053,KeyPers_Berlin_1230638,Name638,FistName638,2024-11-05 10:21:49,PC fixe sans ecran,Precision tower 5xxx
BERLIN_MATERIEL_INFO_202411054,KeyPers_Berlin_1233852,Name3852,FistName3852,2024-11-05 16:47:30,PC fixe sans ecran,ZHAN


In [86]:
psdf_mission_berlin=ps.read_csv('/Users/ordi_de_lvga/Documents/nf26_td/projet/BDD_BGES/BDD_BGES_BERLIN/BDD_BGES_BERLIN_MISSION', sep=';',index_col='ID_MISSION')
psdf_mission_berlin.head(5)

,ID_PERSONNEL,NOM_PERSONNEL,PRENOM_PERSONNEL,DATE_MISSION,TYPE_MISSION,VILLE_DEPART,PAYS_DEPART,VILLE_DESTINATION,PAYS_DESTINATION,TRANSPORT,ALLER_RETOUR
ID_MISSION,,,,,,,,,,,
BERLIN_202411110,KeyPers_Berlin_1230238,Name238,FistName238,2024-11-11 13:34:45,Schulung,Berlin,Allemagne,Buenos Aires,Argentina,Avion,oui
BERLIN_202411111,KeyPers_Berlin_1230352,Name352,FistName352,2024-11-11 03:42:11,Schulung,Berlin,Allemagne,Berlin,Allemagne,Transports en commun,oui
BERLIN_202411112,KeyPers_Berlin_1233087,Name3087,FistName3087,2024-11-11 18:32:48,Entwicklung,New-York,USA,Vancouver,Canada,Avion,oui
BERLIN_202411113,KeyPers_Berlin_1234060,Name4060,FistName4060,2024-11-11 22:06:55,Entwicklung,Paris,France,Tokyo,Japan,Avion,oui
BERLIN_202411114,KeyPers_Berlin_1233427,Name3427,FistName3427,2024-11-11 02:12:52,Geschäftstreffen,Berlin,Allemagne,Los Angeles,USA,Avion,oui


**4.导入伦敦数据**

In [87]:
psdf_person_london=ps.read_csv('/Users/ordi_de_lvga/Documents/nf26_td/projet/BDD_BGES/BDD_BGES_LONDON/PERSONNEL_LONDON.txt', sep=';',index_col='ID_PERSONNEL')
psdf_person_london.head(5)

,NOM_PERSONNEL,PRENOM_PERSONNEL,DT_NAISS,VILLE_NAISS,PAYS_NAISS,NUM_SECU,IND_PAYS_NUM_TELP,NUM_TELEPHONE,NUM_VOIE,DSC_VOIE,CMPL_VOIE,CD_POSTAL,VILLE,PAYS,FONCTION_PERSONNEL,TS_CREATION_PERSONNEL,TS_MAJ_PPERSONNEL
ID_PERSONNEL,,,,,,,,,,,,,,,,,
KeyPers_London_1230000,Name0,FistName0,2004-07-14,Melbourne,Australia,NS000000000,None,+336##0612365,97,NomVoie120,None,#1819,London,England,Data Engineer,2020-10-03 17:41:19,2020-10-03 17:41:19
KeyPers_London_1230001,Name1,FistName1,1954-10-23,Tunis,Tunisie,NS000000001,None,+336##0487142,65,NomVoie351,None,#1748,London,England,Data Engineer,2022-09-09 06:06:04,2022-09-09 06:06:04
KeyPers_London_1230002,Name2,FistName2,1937-04-09,Tokyo,Japan,NS000000002,None,+336##0059989,71,NomVoie442,None,#0669,London,England,Computer Engineer,2007-08-27 10:18:05,2007-08-27 10:18:05
KeyPers_London_1230003,Name3,FistName3,1970-08-07,Bordeaux,France,NS000000003,None,+336##0366753,10,NomVoie743,None,#9255,London,England,Data Engineer,2023-10-02 21:57:57,2023-10-02 21:57:57
KeyPers_London_1230004,Name4,FistName4,1994-11-26,Helsinki,Finlande,NS000000004,None,+336##0422772,47,NomVoie314,None,#7467,London,England,Computer Engineer,2013-01-24 14:22:05,2013-01-24 14:22:05


In [88]:
psdf_informatique_london=ps.read_csv('/Users/ordi_de_lvga/Documents/nf26_td/projet/BDD_BGES/BDD_BGES_LONDON/BDD_BGES_LONDON_INFORMATIQUE', sep=';',index_col='ID_MATERIELINFO')
psdf_informatique_london.head(5)

,ID_PERSONNEL,NOM_PERSONNEL,PRENOM_PERSONNEL,DATE_ACHAT,TYPE,MODELE
ID_MATERIELINFO,,,,,,
LONDON_MATERIEL_INFO_202410030,KeyPers_London_1231533,Name1533,FistName1533,2024-10-03 16:48:26,,"Prodesk (Tower, SFF)"
LONDON_MATERIEL_INFO_202410031,KeyPers_London_1231342,Name1342,FistName1342,2024-10-03 09:05:51,PC portable,ProBook
LONDON_MATERIEL_INFO_202410032,KeyPers_London_1232922,Name2922,FistName2922,2024-10-03 15:55:28,PC fixe sans ecran,Wyse thin client
LONDON_MATERIEL_INFO_202410033,KeyPers_London_1233394,Name3394,FistName3394,2024-10-03 10:38:04,,Latitude 5xxx
LONDON_MATERIEL_INFO_202410034,KeyPers_London_1233566,Name3566,FistName3566,2024-10-03 12:11:29,PC portable,MacBook air pre-retina


In [89]:
psdf_mission_london=ps.read_csv('/Users/ordi_de_lvga/Documents/nf26_td/projet/BDD_BGES/BDD_BGES_LONDON/BDD_BGES_LONDON_MISSION', sep=';',index_col='ID_MISSION')
psdf_mission_london.head(5)

,ID_PERSONNEL,NOM_PERSONNEL,PRENOM_PERSONNEL,DATE_MISSION,TYPE_MISSION,VILLE_DEPART,PAYS_DEPART,VILLE_DESTINATION,PAYS_DESTINATION,TRANSPORT,ALLER_RETOUR
ID_MISSION,,,,,,,,,,,
LONDON_202410190,KeyPers_London_1231718,Name1718,FistName1718,2024-10-19 01:15:13,Vocational Training,London,England,Melbourne,Australia,Avion,non
LONDON_202410191,KeyPers_London_1230403,Name403,FistName403,2024-10-19 12:03:35,Development,London,England,Alger,Algeria,Avion,oui
LONDON_202410192,KeyPers_London_1233125,Name3125,FistName3125,2024-10-19 03:22:25,Vocational Training,London,England,Montreal,Canada,Avion,oui
LONDON_202410193,KeyPers_London_1231247,Name1247,FistName1247,2024-10-19 09:20:18,Team Meeting,London,England,Rabat,Maroc,Avion,oui
LONDON_202410194,KeyPers_London_1231939,Name1939,FistName1939,2024-10-19 06:26:02,Development,London,England,Los Angeles,USA,Avion,oui


**5.导入洛杉矶数据**

In [90]:
psdf_person_losangeles=ps.read_csv('/Users/ordi_de_lvga/Documents/nf26_td/projet/BDD_BGES/BDD_BGES_LOSANGELES/PERSONNEL_LOSANGELES.txt', sep=';',index_col='ID_PERSONNEL')
psdf_person_losangeles.head(5)

,NOM_PERSONNEL,PRENOM_PERSONNEL,DT_NAISS,VILLE_NAISS,PAYS_NAISS,NUM_SECU,IND_PAYS_NUM_TELP,NUM_TELEPHONE,NUM_VOIE,DSC_VOIE,CMPL_VOIE,CD_POSTAL,VILLE,PAYS,FONCTION_PERSONNEL,TS_CREATION_PERSONNEL,TS_MAJ_PPERSONNEL
ID_PERSONNEL,,,,,,,,,,,,,,,,,
KeyPers_LA_1230000,Name0,FistName0,2012-03-20,Compiègne,France,NS000000000,None,+336##0864857,80,NomVoie730,None,#8378,Los Angeles,USA,Business Executive,2011-01-20 04:14:27,2011-01-20 04:14:27
KeyPers_LA_1230001,Name1,FistName1,1943-11-04,Auckland,New Zealand,NS000000001,None,+336##0389143,72,NomVoie51,None,#5530,Los Angeles,USA,Data Engineer,2001-04-12 16:39:19,2001-04-12 16:39:19
KeyPers_LA_1230002,Name2,FistName2,1993-10-25,Paris,France,NS000000002,None,+336##0553330,6,NomVoie112,None,#5279,Los Angeles,USA,Economist,2010-10-21 15:52:41,2010-10-21 15:52:41
KeyPers_LA_1230003,Name3,FistName3,1983-02-27,Washington,USA,NS000000003,None,+336##0860769,18,NomVoie503,None,#5909,Los Angeles,USA,Data Engineer,2021-08-09 11:11:53,2021-08-09 11:11:53
KeyPers_LA_1230004,Name4,FistName4,2013-11-05,London,England,NS000000004,None,+336##0791470,23,NomVoie454,None,#2837,Los Angeles,USA,Data Engineer,2022-05-23 18:50:27,2022-05-23 18:50:27


In [91]:
psdf_informatique_losangeles=ps.read_csv('/Users/ordi_de_lvga/Documents/nf26_td/projet/BDD_BGES/BDD_BGES_LOSANGELES/BDD_BGES_LOSANGELES_INFORMATIQUE', sep=';',index_col='ID_MATERIELINFO')
psdf_informatique_losangeles.head(5)

,ID_PERSONNEL,NOM_PERSONNEL,PRENOM_PERSONNEL,DATE_ACHAT,TYPE,MODELE
ID_MATERIELINFO,,,,,,
LA_MATERIEL_INFO_202407060,KeyPers_LA_1231978,Name1978,FistName1978,2024-07-06 08:39:05,Vidéo projecteur,modèle par défaut
LA_MATERIEL_INFO_202407061,KeyPers_LA_1231609,Name1609,FistName1609,2024-07-06 08:10:36,PC fixe sans ecran,Optiplex small
LA_MATERIEL_INFO_202407062,KeyPers_LA_1230112,Name112,FistName112,2024-07-06 13:35:18,imprimante,Laser A3 (>100kg)
LA_MATERIEL_INFO_202407063,KeyPers_LA_1232436,Name2436,FistName2436,2024-07-06 15:32:55,Serveur,Modèle par défaut
LA_MATERIEL_INFO_202407064,KeyPers_LA_1232904,Name2904,FistName2904,2024-07-06 10:21:24,PC fixe sans ecran,"EliteDesk (Tower, SFF, One)"


In [92]:
psdf_mission_losangeles=ps.read_csv('/Users/ordi_de_lvga/Documents/nf26_td/projet/BDD_BGES/BDD_BGES_LOSANGELES/BDD_BGES_LOSANGELES_MISSION', sep=';',index_col='ID_MISSION')
psdf_mission_losangeles.head(5)

,ID_PERSONNEL,NOM_PERSONNEL,PRENOM_PERSONNEL,DATE_MISSION,TYPE_MISSION,VILLE_DEPART,PAYS_DEPART,VILLE_DESTINATION,PAYS_DESTINATION,TRANSPORT,ALLER_RETOUR
ID_MISSION,,,,,,,,,,,
LA_202405070,KeyPers_LA_1232039,Name2039,FistName2039,2024-05-07 07:03:44,Development,Los Angeles,USA,Lille,France,Avion,oui
LA_202405071,KeyPers_LA_1232529,Name2529,FistName2529,2024-05-07 22:57:21,Vocational Training,Los Angeles,USA,Los Angeles,USA,Transports en commun,non
LA_202405072,KeyPers_LA_1231329,Name1329,FistName1329,2024-05-07 06:27:09,Conference,Los Angeles,USA,Sao Paulo,Brazil,Avion,oui
LA_202405073,KeyPers_LA_1230475,Name475,FistName475,2024-05-07 12:06:31,Team Meeting,Paris,France,Bogota,Colombia,Avion,oui
LA_202405074,KeyPers_LA_1230188,Name188,FistName188,2024-05-07 19:41:53,Conference,New-York,USA,Rabat,Maroc,Avion,oui


**6.导入纽约数据**

In [93]:
psdf_person_newyork=ps.read_csv('/Users/ordi_de_lvga/Documents/nf26_td/projet/BDD_BGES/BDD_BGES_NEWYORK/PERSONNEL_NEWYORK.txt', sep=';',index_col='ID_PERSONNEL')
psdf_person_newyork.head(5)

,NOM_PERSONNEL,PRENOM_PERSONNEL,DT_NAISS,VILLE_NAISS,PAYS_NAISS,NUM_SECU,IND_PAYS_NUM_TELP,NUM_TELEPHONE,NUM_VOIE,DSC_VOIE,CMPL_VOIE,CD_POSTAL,VILLE,PAYS,FONCTION_PERSONNEL,TS_CREATION_PERSONNEL,TS_MAJ_PPERSONNEL
ID_PERSONNEL,,,,,,,,,,,,,,,,,
KeyPers_NewYork_1230000,Name0,FistName0,2011-06-01,Tokyo,Japan,NS000000000,None,+336##0264749,33,NomVoie790,None,#9358,New-York,USA,Business Executive,2021-07-10 15:48:40,2021-07-10 15:48:40
KeyPers_NewYork_1230001,Name1,FistName1,1995-04-10,Oslo,Norvège,NS000000001,None,+336##0035150,94,NomVoie871,None,#1775,New-York,USA,Computer Engineer,2013-11-04 14:06:40,2013-11-04 14:06:40
KeyPers_NewYork_1230002,Name2,FistName2,1963-04-24,Stockholm,Suède,NS000000002,None,+336##0076311,31,NomVoie472,None,#4431,New-York,USA,Computer Engineer,2020-10-20 11:36:07,2020-10-20 11:36:07
KeyPers_NewYork_1230003,Name3,FistName3,1977-09-03,Osaka,Japan,NS000000003,None,+336##0151475,41,NomVoie593,None,#9525,New-York,USA,Data Engineer,2012-10-27 03:07:45,2012-10-27 03:07:45
KeyPers_NewYork_1230004,Name4,FistName4,1950-06-17,Mexico,Mexico,NS000000004,None,+336##0394564,0,NomVoie794,None,#3464,New-York,USA,Computer Engineer,2019-03-21 11:07:16,2019-03-21 11:07:16


In [94]:
psdf_informatique_newyork=ps.read_csv('/Users/ordi_de_lvga/Documents/nf26_td/projet/BDD_BGES/BDD_BGES_NEWYORK/BDD_BGES_NEWYORK_INFORMATIQUE', sep=';',index_col='ID_MATERIELINFO')
psdf_informatique_newyork.head(5)

,ID_PERSONNEL,NOM_PERSONNEL,PRENOM_PERSONNEL,DATE_ACHAT,TYPE,MODELE
ID_MATERIELINFO,,,,,,
NewYork_MATERIEL_INFO_202406250,KeyPers_NewYork_1231314,Name1314,FistName1314,2024-06-25 09:01:53,PC fixe sans ecran,Station de travail (moy)
NewYork_MATERIEL_INFO_202406251,KeyPers_NewYork_1231458,Name1458,FistName1458,2024-06-25 09:41:42,Station d'accueil,modèle par défaut
NewYork_MATERIEL_INFO_202406252,KeyPers_NewYork_1231652,Name1652,FistName1652,2024-06-25 15:21:36,PC fixe tout-en-un,Mac pro
NewYork_MATERIEL_INFO_202406253,KeyPers_NewYork_1231180,Name1180,FistName1180,2024-06-25 08:28:16,PC fixe sans ecran,Precision tower 3xxx
NewYork_MATERIEL_INFO_202406254,KeyPers_NewYork_1230731,Name731,FistName731,2024-06-25 14:04:36,PC fixe sans ecran,Wyse thin client


In [95]:
psdf_mission_newyork=ps.read_csv('/Users/ordi_de_lvga/Documents/nf26_td/projet/BDD_BGES/BDD_BGES_NEWYORK/BDD_BGES_NEWYORK_MISSION', sep=';',index_col='ID_MISSION')
psdf_mission_newyork.head(5)

,ID_PERSONNEL,NOM_PERSONNEL,PRENOM_PERSONNEL,DATE_MISSION,TYPE_MISSION,VILLE_DEPART,PAYS_DEPART,VILLE_DESTINATION,PAYS_DESTINATION,TRANSPORT,ALLER_RETOUR
ID_MISSION,,,,,,,,,,,
NewYork_202405010,KeyPers_NewYork_1230589,Name589,FistName589,2024-05-01 02:03:41,Conference,New-York,USA,Auckland,New Zealand,Avion,oui
NewYork_202405011,KeyPers_NewYork_1231060,Name1060,FistName1060,2024-05-01 21:53:59,Business Meeting,New-York,USA,Tunis,Tunisie,Avion,oui
NewYork_202405012,KeyPers_NewYork_1230977,Name977,FistName977,2024-05-01 02:25:02,Vocational Training,New-York,USA,Sao Paulo,Brazil,Avion,oui
NewYork_202405013,KeyPers_NewYork_1230284,Name284,FistName284,2024-05-01 05:31:53,Business Meeting,New-York,USA,Pekin,China,Avion,oui
NewYork_202405014,KeyPers_NewYork_1232661,Name2661,FistName2661,2024-05-01 05:55:53,Team Meeting,New-York,USA,London,England,Avion,oui


**转换为spark类数据**

In [96]:
sdf_person_paris=psdf_person_paris.to_spark()
sdf_informatique_paris=psdf_informatique_paris.to_spark()
sdf_mission_paris=psdf_mission_paris.to_spark()

sdf_person_shanghai=psdf_person_shanghai.to_spark()
sdf_informatique_shanghai=psdf_informatique_shanghai.to_spark()
sdf_mission_shanghai=psdf_mission_shanghai.to_spark()

sdf_person_berlin=psdf_person_berlin.to_spark()
sdf_informatique_berlin=psdf_informatique_berlin.to_spark()
sdf_mission_berlin=psdf_mission_berlin.to_spark()

sdf_person_london=psdf_person_london.to_spark()
sdf_informatique_london=psdf_informatique_london.to_spark() 
sdf_mission_london=psdf_mission_london.to_spark()

sdf_person_losangeles=psdf_person_losangeles.to_spark()
sdf_informatique_losangeles=psdf_informatique_losangeles.to_spark()
sdf_mission_losangeles=psdf_mission_losangeles.to_spark()

sdf_person_newyork=psdf_person_newyork.to_spark()
sdf_informatique_newyork=psdf_informatique_newyork.to_spark()
sdf_mission_newyork=psdf_mission_newyork.to_spark()



/opt/anaconda3/envs/pyspark_env/lib/python3.12/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


**ETL_person**

In [97]:
sdf_person_paris.show(2)
print('before replace')
sdf_person_paris_1=sdf_person_paris.select(sdf_person_paris.FONCTION_PERSONNEL).distinct().show()
sdf_person_paris=sdf_person_paris.replace({'Ingénieur Data':'Data Engineer','Economiste':'Economist','DRH':'HRD','Ingénieur Informaticien':'Computer Engineer','Cadre':'Business Executive'},subset=['FONCTION_PERSONNEL'])
print('after replace')
sdf_person_paris_1=sdf_person_paris.select(sdf_person_paris.FONCTION_PERSONNEL).distinct().show()

+-------------+----------------+----------+-----------+----------+-----------+-----------------+-------------+--------+----------+---------+---------+-----+------+--------------------+---------------------+-------------------+
|NOM_PERSONNEL|PRENOM_PERSONNEL|  DT_NAISS|VILLE_NAISS|PAYS_NAISS|   NUM_SECU|IND_PAYS_NUM_TELP|NUM_TELEPHONE|NUM_VOIE|  DSC_VOIE|CMPL_VOIE|CD_POSTAL|VILLE|  PAYS|  FONCTION_PERSONNEL|TS_CREATION_PERSONNEL|  TS_MAJ_PPERSONNEL|
+-------------+----------------+----------+-----------+----------+-----------+-----------------+-------------+--------+----------+---------+---------+-----+------+--------------------+---------------------+-------------------+
|        Name0|       FistName0|1992-03-13|      Lille|    France|NS000000000|             NULL|+336##0151713|      22|NomVoie100|     NULL|    #9423|Paris|France|Ingénieur Informa...|  2005-06-24 06:48:21|2005-06-24 06:48:21|
|        Name1|       FistName1|1955-02-17|      Pekin|     China|NS000000001|             N

In [98]:
sdf_person_shanghai.show(3)
sdf_person_shanghai_1=sdf_person_shanghai.select(sdf_person_shanghai.FONCTION_PERSONNEL).distinct().show(20)
print('no need to replace')


+-------------+----------------+----------+--------------+----------+-----------+-----------------+-------------+--------+----------+---------+---------+--------+-----+------------------+---------------------+-------------------+
|NOM_PERSONNEL|PRENOM_PERSONNEL|  DT_NAISS|   VILLE_NAISS|PAYS_NAISS|   NUM_SECU|IND_PAYS_NUM_TELP|NUM_TELEPHONE|NUM_VOIE|  DSC_VOIE|CMPL_VOIE|CD_POSTAL|   VILLE| PAYS|FONCTION_PERSONNEL|TS_CREATION_PERSONNEL|  TS_MAJ_PPERSONNEL|
+-------------+----------------+----------+--------------+----------+-----------+-----------------+-------------+--------+----------+---------+---------+--------+-----+------------------+---------------------+-------------------+
|        Name0|       FistName0|2001-05-14|Rio de Janeiro|    Brazil|NS000000000|             NULL|+336##0481414|      86|NomVoie440|     NULL|    #4474|Shanghai|China|Business Executive|  2015-07-07 10:18:57|2015-07-07 10:18:57|
|        Name1|       FistName1|1995-06-01|     Compiègne|    France|NS000000001

In [99]:
sdf_person_berlin.show(3)
print('before replace')
sdf_person_berlin_1=sdf_person_berlin.select(sdf_person_berlin.FONCTION_PERSONNEL).distinct().show()
print('after replace')
sdf_person_berlin=sdf_person_paris.replace({'Dateningenieur':'Data Engineer','Ökonom':'Economist','Personalleiter':'HRD','Computeringenieur':'Computer Engineer','Führungskraft':'Business Executive'},subset=['FONCTION_PERSONNEL'])
sdf_person_berlin.select(sdf_person_berlin.FONCTION_PERSONNEL).distinct().show()

+-------------+----------------+----------+-----------+-----------+-----------+-----------------+-------------+--------+----------+---------+---------+------+-------+------------------+---------------------+-------------------+
|NOM_PERSONNEL|PRENOM_PERSONNEL|  DT_NAISS|VILLE_NAISS| PAYS_NAISS|   NUM_SECU|IND_PAYS_NUM_TELP|NUM_TELEPHONE|NUM_VOIE|  DSC_VOIE|CMPL_VOIE|CD_POSTAL| VILLE|   PAYS|FONCTION_PERSONNEL|TS_CREATION_PERSONNEL|  TS_MAJ_PPERSONNEL|
+-------------+----------------+----------+-----------+-----------+-----------+-----------------+-------------+--------+----------+---------+---------+------+-------+------------------+---------------------+-------------------+
|        Name0|       FistName0|1993-11-04|      Osaka|      Japan|NS000000000|             NULL|+336##0263188|      57|NomVoie940|     NULL|    #8830|Berlin|Germany|    Dateningenieur|  2010-09-09 11:22:17|2010-09-09 11:22:17|
|        Name1|       FistName1|1932-11-22| Wellington|New Zealand|NS000000001|         

In [100]:
sdf_person_london.show(3)
sdf_person_london_1=sdf_person_london.select(sdf_person_london.FONCTION_PERSONNEL).distinct().show(20)
print('no need to replace')

+-------------+----------------+----------+-----------+----------+-----------+-----------------+-------------+--------+----------+---------+---------+------+-------+------------------+---------------------+-------------------+
|NOM_PERSONNEL|PRENOM_PERSONNEL|  DT_NAISS|VILLE_NAISS|PAYS_NAISS|   NUM_SECU|IND_PAYS_NUM_TELP|NUM_TELEPHONE|NUM_VOIE|  DSC_VOIE|CMPL_VOIE|CD_POSTAL| VILLE|   PAYS|FONCTION_PERSONNEL|TS_CREATION_PERSONNEL|  TS_MAJ_PPERSONNEL|
+-------------+----------------+----------+-----------+----------+-----------+-----------------+-------------+--------+----------+---------+---------+------+-------+------------------+---------------------+-------------------+
|        Name0|       FistName0|2004-07-14|  Melbourne| Australia|NS000000000|             NULL|+336##0612365|      97|NomVoie120|     NULL|    #1819|London|England|     Data Engineer|  2020-10-03 17:41:19|2020-10-03 17:41:19|
|        Name1|       FistName1|1954-10-23|      Tunis|   Tunisie|NS000000001|             N

In [101]:
sdf_person_losangeles.show(3)
sdf_person_losangeles_1=sdf_person_losangeles.select(sdf_person_losangeles.FONCTION_PERSONNEL).distinct().show(20)
print('no need to replace')

+-------------+----------------+----------+-----------+-----------+-----------+-----------------+-------------+--------+----------+---------+---------+-----------+----+------------------+---------------------+-------------------+
|NOM_PERSONNEL|PRENOM_PERSONNEL|  DT_NAISS|VILLE_NAISS| PAYS_NAISS|   NUM_SECU|IND_PAYS_NUM_TELP|NUM_TELEPHONE|NUM_VOIE|  DSC_VOIE|CMPL_VOIE|CD_POSTAL|      VILLE|PAYS|FONCTION_PERSONNEL|TS_CREATION_PERSONNEL|  TS_MAJ_PPERSONNEL|
+-------------+----------------+----------+-----------+-----------+-----------+-----------------+-------------+--------+----------+---------+---------+-----------+----+------------------+---------------------+-------------------+
|        Name0|       FistName0|2012-03-20|  Compiègne|     France|NS000000000|             NULL|+336##0864857|      80|NomVoie730|     NULL|    #8378|Los Angeles| USA|Business Executive|  2011-01-20 04:14:27|2011-01-20 04:14:27|
|        Name1|       FistName1|1943-11-04|   Auckland|New Zealand|NS000000001| 

In [102]:
sdf_person_newyork.show(3)
sdf_person_newyork_1=sdf_person_newyork.select(sdf_person_newyork.FONCTION_PERSONNEL).distinct().show(20)
print('no need to replace')

+-------------+----------------+----------+-----------+----------+-----------+-----------------+-------------+--------+----------+---------+---------+--------+----+------------------+---------------------+-------------------+
|NOM_PERSONNEL|PRENOM_PERSONNEL|  DT_NAISS|VILLE_NAISS|PAYS_NAISS|   NUM_SECU|IND_PAYS_NUM_TELP|NUM_TELEPHONE|NUM_VOIE|  DSC_VOIE|CMPL_VOIE|CD_POSTAL|   VILLE|PAYS|FONCTION_PERSONNEL|TS_CREATION_PERSONNEL|  TS_MAJ_PPERSONNEL|
+-------------+----------------+----------+-----------+----------+-----------+-----------------+-------------+--------+----------+---------+---------+--------+----+------------------+---------------------+-------------------+
|        Name0|       FistName0|2011-06-01|      Tokyo|     Japan|NS000000000|             NULL|+336##0264749|      33|NomVoie790|     NULL|    #9358|New-York| USA|Business Executive|  2021-07-10 15:48:40|2021-07-10 15:48:40|
|        Name1|       FistName1|1995-04-10|       Oslo|   Norvège|NS000000001|             NULL|

**ETL_MISSION**

In [103]:
sdf_mission_paris.show(3)
print('before replace')
sdf_mission_paris_1=sdf_mission_paris.select(sdf_mission_paris.TYPE_MISSION).distinct().show(20)
print('after replace')
sdf_mission_paris=sdf_mission_paris.replace({'Conférence':'Conference','Réunion':'Team Meeting','Développement':'Development','Formation':'Vocational Training','Rencontre entreprises':'Business Meeting'},subset=['TYPE_MISSION'])
sdf_mission_paris.select(sdf_mission_paris.TYPE_MISSION).distinct().show(20)

+--------------------+-------------+----------------+-------------------+-------------+------------+-----------+-----------------+----------------+---------+------------+
|        ID_PERSONNEL|NOM_PERSONNEL|PRENOM_PERSONNEL|       DATE_MISSION| TYPE_MISSION|VILLE_DEPART|PAYS_DEPART|VILLE_DESTINATION|PAYS_DESTINATION|TRANSPORT|ALLER_RETOUR|
+--------------------+-------------+----------------+-------------------+-------------+------------+-----------+-----------------+----------------+---------+------------+
|KeyPers_Paris_123...|     Name3185|    FistName3185|2024-05-18 23:19:11|      Réunion|       Paris|     France|       Wellington|     New Zealand|    Avion|         oui|
|KeyPers_Paris_123...|      Name183|     FistName183|2024-05-18 10:54:36|Développement|       Paris|     France|           Sidney|       Australia|    Avion|         oui|
|KeyPers_Paris_123...|     Name2953|    FistName2953|2024-05-18 06:43:42|Développement|       Paris|     France|         Auckland|     New Zealan

In [104]:
sdf_mission_shanghai.show(3)
sdf_mission_shanghai_1=sdf_mission_shanghai.select(sdf_mission_shanghai.TYPE_MISSION).distinct().show(20)
print('no need to replace')

+--------------------+-------------+----------------+-------------------+-------------------+------------+-----------+-----------------+----------------+---------+------------+
|        ID_PERSONNEL|NOM_PERSONNEL|PRENOM_PERSONNEL|       DATE_MISSION|       TYPE_MISSION|VILLE_DEPART|PAYS_DEPART|VILLE_DESTINATION|PAYS_DESTINATION|TRANSPORT|ALLER_RETOUR|
+--------------------+-------------+----------------+-------------------+-------------------+------------+-----------+-----------------+----------------+---------+------------+
|KeyPers_Shanghai_...|      Name528|     FistName528|2024-06-05 15:10:01|        Development|    Shanghai|      China|      Los Angeles|             USA|    Avion|         oui|
|KeyPers_Shanghai_...|      Name794|     FistName794|2024-06-05 08:12:49|Vocational Training|    Shanghai|      China|            Dubaï|         Emirats|    Avion|         oui|
|KeyPers_Shanghai_...|      Name681|     FistName681|2024-06-05 09:43:46|        Development|    Shanghai|      Chi

In [105]:
sdf_mission_berlin.show(3)
print('before replace')
sdf_mission_berlin_1=sdf_mission_berlin.select(sdf_mission_berlin.TYPE_MISSION).distinct().show(20)
print('after replace')
sdf_mission_berlin=sdf_mission_berlin.replace({'Geschäftstreffen':'Business Meeting','Konferenz':'Conference','Schulung':'Vocational Training','Meeting':'Team Meeting','Entwicklung':'Development'},subset=['TYPE_MISSION'])
sdf_mission_berlin.select(sdf_mission_berlin.TYPE_MISSION).distinct().show()

+--------------------+-------------+----------------+-------------------+------------+------------+-----------+-----------------+----------------+--------------------+------------+
|        ID_PERSONNEL|NOM_PERSONNEL|PRENOM_PERSONNEL|       DATE_MISSION|TYPE_MISSION|VILLE_DEPART|PAYS_DEPART|VILLE_DESTINATION|PAYS_DESTINATION|           TRANSPORT|ALLER_RETOUR|
+--------------------+-------------+----------------+-------------------+------------+------------+-----------+-----------------+----------------+--------------------+------------+
|KeyPers_Berlin_12...|      Name238|     FistName238|2024-11-11 13:34:45|    Schulung|      Berlin|  Allemagne|     Buenos Aires|       Argentina|               Avion|         oui|
|KeyPers_Berlin_12...|      Name352|     FistName352|2024-11-11 03:42:11|    Schulung|      Berlin|  Allemagne|           Berlin|       Allemagne|Transports en commun|         oui|
|KeyPers_Berlin_12...|     Name3087|    FistName3087|2024-11-11 18:32:48| Entwicklung|    New-Y

In [106]:
sdf_mission_london.show(3)
sdf_mission_london_1=sdf_mission_london.select(sdf_mission_london.TYPE_MISSION).distinct().show(20)
print('no need to replace')

+--------------------+-------------+----------------+-------------------+-------------------+------------+-----------+-----------------+----------------+---------+------------+
|        ID_PERSONNEL|NOM_PERSONNEL|PRENOM_PERSONNEL|       DATE_MISSION|       TYPE_MISSION|VILLE_DEPART|PAYS_DEPART|VILLE_DESTINATION|PAYS_DESTINATION|TRANSPORT|ALLER_RETOUR|
+--------------------+-------------+----------------+-------------------+-------------------+------------+-----------+-----------------+----------------+---------+------------+
|KeyPers_London_12...|     Name1718|    FistName1718|2024-10-19 01:15:13|Vocational Training|      London|    England|        Melbourne|       Australia|    Avion|         non|
|KeyPers_London_12...|      Name403|     FistName403|2024-10-19 12:03:35|        Development|      London|    England|            Alger|         Algeria|    Avion|         oui|
|KeyPers_London_12...|     Name3125|    FistName3125|2024-10-19 03:22:25|Vocational Training|      London|    Engla

In [107]:
sdf_mission_losangeles.show(3)
sdf_mission_losangeles_1=sdf_mission_losangeles.select(sdf_mission_losangeles.TYPE_MISSION).distinct().show(20)
print('no need to replace')

+------------------+-------------+----------------+-------------------+-------------------+------------+-----------+-----------------+----------------+--------------------+------------+
|      ID_PERSONNEL|NOM_PERSONNEL|PRENOM_PERSONNEL|       DATE_MISSION|       TYPE_MISSION|VILLE_DEPART|PAYS_DEPART|VILLE_DESTINATION|PAYS_DESTINATION|           TRANSPORT|ALLER_RETOUR|
+------------------+-------------+----------------+-------------------+-------------------+------------+-----------+-----------------+----------------+--------------------+------------+
|KeyPers_LA_1232039|     Name2039|    FistName2039|2024-05-07 07:03:44|        Development| Los Angeles|        USA|            Lille|          France|               Avion|         oui|
|KeyPers_LA_1232529|     Name2529|    FistName2529|2024-05-07 22:57:21|Vocational Training| Los Angeles|        USA|      Los Angeles|             USA|Transports en commun|         non|
|KeyPers_LA_1231329|     Name1329|    FistName1329|2024-05-07 06:27:09

In [108]:
sdf_mission_newyork.show(3)
sdf_mission_newyork_1=sdf_mission_newyork.select(sdf_mission_newyork.TYPE_MISSION).distinct().show(20)
print('no need to replace')

+--------------------+-------------+----------------+-------------------+-------------------+------------+-----------+-----------------+----------------+---------+------------+
|        ID_PERSONNEL|NOM_PERSONNEL|PRENOM_PERSONNEL|       DATE_MISSION|       TYPE_MISSION|VILLE_DEPART|PAYS_DEPART|VILLE_DESTINATION|PAYS_DESTINATION|TRANSPORT|ALLER_RETOUR|
+--------------------+-------------+----------------+-------------------+-------------------+------------+-----------+-----------------+----------------+---------+------------+
|KeyPers_NewYork_1...|      Name589|     FistName589|2024-05-01 02:03:41|         Conference|    New-York|        USA|         Auckland|     New Zealand|    Avion|         oui|
|KeyPers_NewYork_1...|     Name1060|    FistName1060|2024-05-01 21:53:59|   Business Meeting|    New-York|        USA|            Tunis|         Tunisie|    Avion|         oui|
|KeyPers_NewYork_1...|      Name977|     FistName977|2024-05-01 02:25:02|Vocational Training|    New-York|        U

**GEOPY表示距离**

In [109]:
from geopy import distance
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="NF26_UTC",timeout=10)
location_p = geolocator.geocode("Paris, France")
print((location_p.latitude, location_p.longitude))
location_l = geolocator.geocode("Los Angeles, USA")
print((location_l.latitude, location_l.longitude))
location_s = geolocator.geocode("Shanghai, China")
print((location_s.latitude, location_s.longitude))
location_n = geolocator.geocode("New York, USA")
print((location_n.latitude, location_n.longitude))
location_b = geolocator.geocode("Berlin, Allemagne")
print((location_b.latitude, location_b.longitude))
location_london = geolocator.geocode("London, England")
print((location_london.latitude, location_london.longitude))



(48.8588897, 2.3200410217200766)
(34.0536909, -118.242766)
(31.2312707, 121.4700152)
(40.7127281, -74.0060152)
(52.510885, 13.3989367)
(51.4893335, -0.14405508452768728)


In [110]:
def get_lat_lon(city, country,):
    try:
        location = geolocator.geocode(f"{city}, {country}")
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Error retrieving coordinates for {city}, {country}: {e}")
        return None, None
sdf_lat_lon_paris = sdf_mission_paris.select('VILLE_DEPART','PAYS_DEPART','VILLE_DESTINATION','PAYS_DESTINATION','TRANSPORT','ALLER_RETOUR')
psdf_lat_lon_paris = sdf_lat_lon_paris.toPandas()
psdf_lat_lon_paris.head(3)
sdf0=sdf_mission_paris.select('VILLE_DEPART','PAYS_DEPART').distinct()
psdf0 = sdf0.toPandas()
results_depart=psdf0.apply(lambda row:pd.Series(get_lat_lon(row['VILLE_DEPART'], row['PAYS_DEPART'])), axis=1)
print(results_depart)



           0           1
0  34.053691 -118.242766
1  48.858890    2.320041
2  31.231271  121.470015
3  40.712728  -74.006015
4  51.489334   -0.144055
5  52.510885   13.398937


In [111]:
psdf0[['latitude', 'longitude']] = results_depart
psdf0.head(3)

,VILLE_DEPART,PAYS_DEPART,latitude,longitude
0,Los Angeles,USA,34.053691,-118.242766
1,Paris,France,48.858890,2.320041
2,Shanghai,China,31.231271,121.470015


In [112]:
print(sdf_mission_paris.select('VILLE_DESTINATION', 'PAYS_DESTINATION').distinct().count())
sdf1 = sdf_mission_paris.select('VILLE_DESTINATION', 'PAYS_DESTINATION').distinct()
psdf1 = sdf1.toPandas()
psdf1.head(3)



32


,VILLE_DESTINATION,PAYS_DESTINATION
0,Dubaï,Emirats
1,Los Angeles,USA
2,Wellington,New Zealand


In [113]:
#####运算时见较长
results_destination=psdf1.apply(lambda row:pd.Series(get_lat_lon(row['VILLE_DESTINATION'], row['PAYS_DESTINATION'])), axis=1)
print(results_destination)


            0           1
0   25.264723   55.292415
1   34.053691 -118.242766
2  -41.288795  174.777211
3   23.658512 -102.007710
4   60.167488   24.942747
5   31.231271  121.470015
6  -36.852095  174.763180
7    4.653382  -74.083633
8   49.260872 -123.113952
9   50.636565    3.063528
10  33.985178   -6.846143
11  59.325117   18.071093
12  49.417950    2.826317
13  51.489334   -0.144055
14  36.772933    3.058845
15  35.676860  139.763895
16 -23.550651  -46.633382
17  45.503182  -73.569806
18  40.190632  116.412144
19  38.895037  -77.036543
20 -22.911014  -43.209373
21 -12.062107  -77.036526
22  43.296174    5.369953
23  36.800207   10.185776
24  52.510885   13.398937
25  40.712728  -74.006015
26  44.841225   -0.580036
27 -34.608370  -58.444058
28  59.913330   10.738970
29  34.693757  135.501454
30 -37.814245  144.963173
31 -33.869844  151.208285


In [114]:
psdf1[['latitude', 'longitude']] = results_destination
psdf1.head(3)

,VILLE_DESTINATION,PAYS_DESTINATION,latitude,longitude
0,Dubaï,Emirats,25.264723,55.292415
1,Los Angeles,USA,34.053691,-118.242766
2,Wellington,New Zealand,-41.288795,174.777211


In [115]:
merged_depart=psdf_lat_lon_paris.merge(psdf0, on=['VILLE_DEPART','PAYS_DEPART'], how='left')
merged_depart=merged_depart[['VILLE_DEPART','PAYS_DEPART','latitude','longitude']]
psdf_lat_lon_paris[['latitude_depart','longitude_depart']]=merged_depart[['latitude','longitude']]
print(psdf_lat_lon_paris.shape[0])
psdf_lat_lon_paris.head(3)


4518


,VILLE_DEPART,PAYS_DEPART,VILLE_DESTINATION,PAYS_DESTINATION,TRANSPORT,ALLER_RETOUR,latitude_depart,longitude_depart
0,Paris,France,Wellington,New Zealand,Avion,oui,48.85889,2.320041
1,Paris,France,Sidney,Australia,Avion,oui,48.85889,2.320041
2,Paris,France,Auckland,New Zealand,Avion,oui,48.85889,2.320041


In [116]:
merged_destination=psdf_lat_lon_paris.merge(psdf1, on=['VILLE_DESTINATION','PAYS_DESTINATION'], how='left')
merged_destination=merged_destination[['VILLE_DESTINATION','PAYS_DESTINATION','latitude','longitude']]
psdf_lat_lon_paris[['latitude_destination','longitude_destination']]=merged_destination[['latitude','longitude']]
print(psdf_lat_lon_paris.shape[0])
psdf_lat_lon_paris.head(3)

4518


,VILLE_DEPART,PAYS_DEPART,VILLE_DESTINATION,PAYS_DESTINATION,TRANSPORT,ALLER_RETOUR,latitude_depart,longitude_depart,latitude_destination,longitude_destination
0,Paris,France,Wellington,New Zealand,Avion,oui,48.85889,2.320041,-41.288795,174.777211
1,Paris,France,Sidney,Australia,Avion,oui,48.85889,2.320041,-33.869844,151.208285
2,Paris,France,Auckland,New Zealand,Avion,oui,48.85889,2.320041,-36.852095,174.763180


In [118]:
psdf_lat_lon_paris['each_distance']=psdf_lat_lon_paris.apply(lambda row: distance.distance((row['latitude_depart'], row['longitude_depart']), (row['latitude_destination'], row['longitude_destination'])).km, axis=1)
psdf_lat_lon_paris.head(3)
psdf_lat_lon_paris['ALLER_RETOUR']=psdf_lat_lon_paris['ALLER_RETOUR'].replace({'oui':2,'non':1})
psdf_lat_lon_paris.head(3)

/var/folders/w9/n00_q7c121n3slw3dpckcykr0000gn/T/ipykernel_35178/2390864490.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  psdf_lat_lon_paris['ALLER_RETOUR']=psdf_lat_lon_paris['ALLER_RETOUR'].replace({'oui':2,'non':1})


,VILLE_DEPART,PAYS_DEPART,VILLE_DESTINATION,PAYS_DESTINATION,TRANSPORT,ALLER_RETOUR,latitude_depart,longitude_depart,latitude_destination,longitude_destination,each_distance
0,Paris,France,Wellington,New Zealand,Avion,2,48.85889,2.320041,-41.288795,174.777211,18980.624226
1,Paris,France,Sidney,Australia,Avion,2,48.85889,2.320041,-33.869844,151.208285,16959.632941
2,Paris,France,Auckland,New Zealand,Avion,2,48.85889,2.320041,-36.852095,174.763180,18538.879977


In [120]:
psdf_lat_lon_paris['total_distance']=psdf_lat_lon_paris['each_distance']*psdf_lat_lon_paris['ALLER_RETOUR']
psdf_lat_lon_paris.head(20)

,VILLE_DEPART,PAYS_DEPART,VILLE_DESTINATION,PAYS_DESTINATION,TRANSPORT,ALLER_RETOUR,latitude_depart,longitude_depart,latitude_destination,longitude_destination,each_distance,total_distance
0,Paris,France,Wellington,New Zealand,Avion,2,48.858890,2.320041,-41.288795,174.777211,18980.624226,37961.248452
1,Paris,France,Sidney,Australia,Avion,2,48.858890,2.320041,-33.869844,151.208285,16959.632941,33919.265882
2,Paris,France,Auckland,New Zealand,Avion,2,48.858890,2.320041,-36.852095,174.763180,18538.879977,37077.759953
3,Paris,France,Oslo,Norvège,Avion,1,48.858890,2.320041,59.913330,10.738970,1344.066053,1344.066053
4,Paris,France,Lima,Peru,Avion,2,48.858890,2.320041,-12.062107,-77.036526,10245.031215,20490.062429
5,Paris,France,Rabat,Maroc,Avion,2,48.858890,2.320041,33.985178,-6.846143,1817.793821,3635.587641
6,Paris,France,Lille,France,Avion,2,48.858890,2.320041,50.636565,3.063528,204.849593,409.699186
7,Paris,France,Buenos Aires,Argentina,Avion,2,48.858890,2.320041,-34.608370,-58.444058,11027.372369,22054.744739
8,Paris,France,Rabat,Maroc,Avion,2,48.858890,2.320041,33.985178,-6.846143,1817.793821,3635.587641
9,Paris,France,Sao Paulo,Brazil,Avion,2,48.858890,2.320041,-23.550651,-46.633382,9376.930481,18753.860963


In [128]:
psdf_final_mission_paris=psdf_lat_lon_paris
psdf_final_mission_paris.head(3)

,VILLE_DEPART,PAYS_DEPART,VILLE_DESTINATION,PAYS_DESTINATION,TRANSPORT,ALLER_RETOUR,latitude_depart,longitude_depart,latitude_destination,longitude_destination,each_distance,total_distance
0,Paris,France,Wellington,New Zealand,Avion,2,48.85889,2.320041,-41.288795,174.777211,18980.624226,37961.248452
1,Paris,France,Sidney,Australia,Avion,2,48.85889,2.320041,-33.869844,151.208285,16959.632941,33919.265882
2,Paris,France,Auckland,New Zealand,Avion,2,48.85889,2.320041,-36.852095,174.763180,18538.879977,37077.759953


In [ ]:
print(psdf_final_mission_paris[psdf_final_mission_paris['TRANSPORT'] == 'Taxi'].shape[0])
psdf_final_mission_paris[psdf_final_mission_paris['TRANSPORT'] == 'Taxi'].head()


60


,VILLE_DEPART,PAYS_DEPART,VILLE_DESTINATION,PAYS_DESTINATION,TRANSPORT,ALLER_RETOUR,latitude_depart,longitude_depart,latitude_destination,longitude_destination,each_distance,total_distance
62,Paris,France,Compiègne,France,Taxi,2,48.85889,2.320041,49.417950,2.826317,72.321066,144.642132
196,Paris,France,Marseille,France,Taxi,2,48.85889,2.320041,43.296174,5.369953,661.647736,1323.295471
210,Paris,France,Lille,France,Taxi,2,48.85889,2.320041,50.636565,3.063528,204.849593,409.699186
244,Paris,France,Compiègne,France,Taxi,2,48.85889,2.320041,49.417950,2.826317,72.321066,144.642132
252,Paris,France,Bordeaux,France,Taxi,2,48.85889,2.320041,44.841225,-0.580036,498.308641,996.617282


In [130]:
psdf_final_mission_paris['TRANSPORT'].unique()

array(['Avion', 'Taxi', 'Train', 'Transports en commun'], dtype=object)

In [133]:
psdf_discion_file = pd.read_csv('/Users/ordi_de_lvga/Documents/nf26_td/projet/GES1point5_transports_factors_20250408.tsv', sep='\t')
psdf_discion_file.head(3)

,category,subcategory,subsubcategory,unit,name,year,co2,ch4,n2o,other,total,uncertainty,ef.unit
0,Transport,Plane,Short haul (< 1000 km),km,"Short haul plane, with contrails",2022,0,0,0,0,0.2586,0.37,kg CO2e/km
1,Transport,Plane,Medium haul (< 1001 - 3500km),km,"Medium haul plane, with contrails",2022,0,0,0,0,0.1875,0.37,kg CO2e/km
2,Transport,Plane,Long haul (> 3500 km),km,"Long haul plane, with contrails",2022,0,0,0,0,0.1520,0.37,kg CO2e/km


In [ ]:
#For air travel, 95 km is added to the distance as the crow flies
#
def ajuster_distance(row):
    base = row['each_distance']
    mode = row['TRANSPORT']  
    
    if mode == 'Avion':
        return base + 95
    elif mode == 'Taxi':
        return base * 1.2
    elif mode == 'Train':
        return base * 1.2
    elif mode == 'Transports en commun':
        return base * 1.5
    
 
psdf['each_distance'] = psdf.apply(ajuster_distance, axis=1)